# import


In [224]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import itertools
from prophet import Prophet
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error, mean_absolute_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pmdarima.preprocessing import FourierFeaturizer
from pmdarima import auto_arima, ARIMA
import matplotlib.pyplot as plt
import holidays
pd.options.plotting.backend = 'plotly'


# data


In [3]:
df_store = pd.read_pickle('data/df_daily.pkl')
df_company = df_store.groupby('date').sum()
train_data = df_company['sales']/1e6
train_data.index.freq = 'D'

In [4]:
train_data.plot(title='Sales TS in Million VND (aggregated data)')

In [5]:
# yearly seasonality
fouri_terms = FourierFeaturizer(365.25, 2)
y_prime, df_fouri = fouri_terms.fit_transform(train_data)
df_fouri.index = y_prime.index


In [6]:
df_fouri.head()

,FOURIER_S365-0,FOURIER_C365-0,FOURIER_S365-1,FOURIER_C365-1
date,,,,
2017-08-07,0.017202,0.999852,0.034398,0.999408
2017-08-08,0.034398,0.999408,0.068755,0.997634
2017-08-09,0.051584,0.998669,0.103031,0.994678
2017-08-10,0.068755,0.997634,0.137185,0.990545
2017-08-11,0.085906,0.996303,0.171177,0.985240


In [7]:
# holiday
ts_holiday = pd.read_pickle('data/holiday.pkl')
print(ts_holiday.index)

DatetimeIndex(['2018-02-15', '2018-02-16', '2018-02-17', '2018-02-18',
               '2018-02-19', '2018-02-20', '2018-05-01', '2019-02-04',
               '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08',
               '2019-02-09', '2019-05-01', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-05-01'],
              dtype='datetime64[ns]', name='date', freq=None)


# model


## ARIMA model

In [8]:
# combine exog. variables
df_exog = pd.concat([df_fouri, ts_holiday], axis=1)
df_exog['holiday'] = df_exog['holiday'].fillna(False).astype('int')

In [9]:
df_exog.head()

,FOURIER_S365-0,FOURIER_C365-0,FOURIER_S365-1,FOURIER_C365-1,holiday
date,,,,,
2017-08-07,0.017202,0.999852,0.034398,0.999408,0
2017-08-08,0.034398,0.999408,0.068755,0.997634,0
2017-08-09,0.051584,0.998669,0.103031,0.994678,0
2017-08-10,0.068755,0.997634,0.137185,0.990545,0
2017-08-11,0.085906,0.996303,0.171177,0.985240,0


In [10]:
# Split the time series as well as exogenous features data into train and test splits
steps_ahead = 92 # 3 months
y_to_train = y_prime.iloc[:-steps_ahead]
y_to_test = y_prime.iloc[-steps_ahead:]

exog_to_train = df_exog.iloc[:-steps_ahead]
exog_to_test = df_exog.iloc[-steps_ahead:]

In [10]:
# Fit model to the level to find common order
arima_model = auto_arima(
    y=y_to_train,
    exogenous=exog_to_train,
    D=1, 
    seasonal=True, m=7 # Weekly seasonality
    )
arima_model

ARIMA(order=(4, 0, 0), scoring_args={}, seasonal_order=(2, 1, 0, 7),
      suppress_warnings=True, with_intercept=False)

In [11]:
# Forecast
arima_y_fitted = arima_model.predict_in_sample(
    X=exog_to_train)
arima_y_forecast = arima_model.predict(
    n_periods=len(y_to_test), exogenous=exog_to_test)
arima_y_forecast = pd.Series(arima_y_forecast,
                             name='forecast',
                             index=y_to_test.index)

In [11]:
# metrics
# in-sample
# mae_IS = round(mean_absolute_error(y_to_train, arima_y_fitted))
# mape_IS = round(mean_absolute_percentage_error(
#     y_to_train, arima_y_fitted), 3)

# out-sample
mae_OOS = round(mean_absolute_error(y_to_test, arima_y_forecast))
mape_OOS = round(mean_absolute_percentage_error(
    y_to_test, arima_y_forecast), 3)

print(f'Company:')
print('Out of sample:',
      f'MAPE: {mape_OOS}',
      f'MAE: {mae_OOS}\n',
      sep='\n')

NameError: name 'arima_y_fitted' is not defined

In [74]:
# print(arima_model)
res_whole_holiday_promo = pd.DataFrame()

In [75]:
# LOOP all stores
for store in df_store['store_id'].unique():  # print(store)
    df_data = df_store[df_store['store_id'] == store].set_index('date')[
        ['sales', 'promo_count']]
    df_data.index.freq = 'D'
    train_data = df_data['sales']/1e6

    # Prepare the fourier terms to model annual seasonality; add as exogenous features to auto_arima
    fouri_terms = FourierFeaturizer(365.25, 2)
    y_prime, df_fouri = fouri_terms.fit_transform(train_data)
    df_fouri.index = y_prime.index

    # combine exog. variables
    df_exog = pd.concat([df_fouri, ts_holiday], axis=1)
    df_exog['holiday'] = df_exog['holiday'].fillna(False).astype('int')
    df_exog.dropna(inplace=True)

    # Split the time series as well as exogenous features data into train and test splits
    y_to_train = y_prime.iloc[:-steps_ahead]
    y_to_test = y_prime.iloc[-steps_ahead:]

    exog_to_train = df_exog.iloc[:-steps_ahead]
    exog_to_test = df_exog.iloc[-steps_ahead:]

    # Fit model to each store in the level
    arima_model_store = ARIMA(
        order=(4, 0, 0),
        seasonal_order=(2, 1, 0, 7),
        # order=arima_model.order,
        # seasonal_order=arima_model.seasonal_order
    ).fit(y_to_train,
          X=exog_to_train
          )

    # Forecast
    # arima_y_fitted = arima_model_store.predict_in_sample(
    #     X=exog_to_train)
    arima_y_forecast = arima_model_store.predict(
        n_periods=len(y_to_test), exogenous=exog_to_test)
    arima_y_forecast = pd.Series(arima_y_forecast,
                                 name='forecast',
                                 index=y_to_test.index)
    # Random walk
    RW_y_forecast = pd.Series(
        y_to_train[-1], name='RW_forecast', index=y_to_test.index)

    # metrics
    # in-sample
    # mae_IS = round(mean_absolute_error(y_to_train, arima_y_fitted))
    # mape_IS = round(mean_absolute_percentage_error(
    #     y_to_train, arima_y_fitted), 3)

    # out-sample
    mae_OOS = round(mean_absolute_error(y_to_test, arima_y_forecast))
    mape_OOS = round(mean_absolute_percentage_error(
        y_to_test, arima_y_forecast), 3)
    RW_mae_OOS = round(mean_absolute_error(y_to_test, RW_y_forecast))
    RW_mape_OOS = round(mean_absolute_percentage_error(
        y_to_test, RW_y_forecast), 3)

    print(f'Store {store}', 
          f'MAPE: {mape_OOS}',
        #   f'MAE: {mae_OOS}\n',
          sep='\n')

    res_whole_holiday_promo = res_whole_holiday_promo.append({
        'store_id': store,
        # 'fc_IS': arima_y_fitted,
        'fc_OOS': arima_y_forecast,
        'fc_RW': RW_y_forecast,
        'arima_mape_OOS': mape_OOS,
        # 'arima_mape_IS': mape_IS,
        'arima_mae_OOS': mae_OOS,
        # 'arima_mae_IS': mae_IS,
        'RW_mape_OOS': RW_mape_OOS,
        'RW_mae_OOS': RW_mae_OOS,
    }, ignore_index=True)

Store 307222
MAPE: 0.428
Store 307244
MAPE: 0.461
Store 307248
MAPE: 0.374
Store 320264
MAPE: 0.52
Store 328165
MAPE: 0.549


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Store 349920
MAPE: 0.507
Store 349924
MAPE: 0.451
Store 349952
MAPE: 0.457
Store 349958
MAPE: 0.448
Store 349962
MAPE: 0.49
Store 349972
MAPE: 0.455


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Store 349978
MAPE: 0.515
Store 349980
MAPE: 0.423
Store 349998
MAPE: 0.396


...

Store 536902
MAPE: 0.895


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



Store 566790
MAPE: 1.099


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Store 566792
MAPE: 0.648


In [76]:
res_whole_holiday_promo.head()

,RW_mae_OOS,RW_mape_OOS,arima_mae_OOS,arima_mape_OOS,fc_OOS,fc_RW,store_id
0,13.0,0.452,13.0,0.428,date 2020-11-01 26.469510 2020-11-02 20....,date 2020-11-01 21.99 2020-11-02 21.99 2...,307222.0
1,10.0,0.454,10.0,0.461,date 2020-11-01 19.930375 2020-11-02 20....,date 2020-11-01 23.4081 2020-11-02 23.40...,307244.0
2,9.0,0.451,7.0,0.374,date 2020-11-01 20.385643 2020-11-02 13....,date 2020-11-01 14.2416 2020-11-02 14.24...,307248.0
3,7.0,0.465,8.0,0.520,date 2020-11-01 19.012270 2020-11-02 13....,date 2020-11-01 14.91545 2020-11-02 14.9...,320264.0
4,41.0,0.594,37.0,0.549,date 2020-11-01 86.226162 2020-11-02 35....,date 2020-11-01 60.16385 2020-11-02 60.1...,328165.0


In [77]:
res_whole_holiday_promo.loc[0, 'fc_OOS']

date
2020-11-01    26.469510
2020-11-02    20.660000
2020-11-03    24.667324
2020-11-04    20.776159
2020-11-05    20.631250
                ...    
2021-01-27    24.564984
2021-01-28    24.088772
2021-01-29    27.625918
2021-01-30    31.919844
2021-01-31    32.793676
Freq: D, Name: forecast, Length: 92, dtype: float64

In [78]:
# res_whole_holiday_promo.to_pickle('results/res_RW_ARIMA_whole_holiday_promo.pkl')
# res_whole_holiday_promo = pd.read_pickle('results/res_RW_ARIMA_whole_holiday_promo.pkl')

## FB Prophet

In [80]:
fb_df = pd.concat([df_company[['sales']], ts_holiday], axis=1)
fb_df['sales'] = fb_df['sales']/1e6
fb_df['holiday'] = fb_df['holiday'].fillna(False).astype('bool')
fb_df = fb_df.reset_index().rename({'date': 'ds', 'sales': 'y'}, axis=1)
fb_train = fb_df.iloc[:-steps_ahead]
fb_test = fb_df.iloc[-steps_ahead:]

In [81]:
fb_test

,ds,y,holiday
1182,2020-11-01,619.011000,False
1183,2020-11-02,264.581400,False
1184,2020-11-03,307.434750,False
1185,2020-11-04,332.188850,False
1186,2020-11-05,335.843400,False
...,...,...,...
1269,2021-01-27,669.639716,False
1270,2021-01-28,720.317779,False
1271,2021-01-29,882.390069,False
1272,2021-01-30,1219.184443,False


In [83]:
# Grid search set up
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

all_params = [dict(zip(param_grid.keys(), v))
              for v in itertools.product(*param_grid.values())]

In [90]:
mape_PROPHET = []
for params in all_params:
    # set up model
    m = Prophet(**params)
    m.add_regressor('holiday')
    m.fit(fb_train)
    # set up CV
    df_cv = cross_validation(
        m,
        initial=(str(fb_train.shape[0]-steps_ahead*2)+' days'),
        period=(str(steps_ahead)+' days'),
        horizon=(str(steps_ahead)+' days'),
        )
    # evaluate
    df_p = performance_metrics(df_cv)
    mape_PROPHET.append(df_p['mape'].values[-1])


# result
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape_PROPHET
tuning_results = tuning_results.sort_values('mape').reset_index(drop=True)
tuning_results


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 1 forecasts with cutoffs between 2020-07-31 00:00:00 and 2020-07-31 00:00:00


Initial log joint probability = -8.2067
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2456.13   7.41044e-07       5103.42      0.1444      0.6373      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     156       2456.22   4.54406e-09       4993.58       0.358       0.358      196   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


Initial log joint probability = -10.8767
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2241.65   1.66047e-08       4456.34      0.4385      0.4385      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     108       2241.65    7.7757e-09       4287.14      0.6344      0.6344      133   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


100%|██████████| 1/1 [00:01<00:00,  1.87s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 1 forecasts with cutoffs between 2020-07-31 00:00:00 and 2020-07-31 00:00:00


Initial log joint probability = -8.2067
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2472.49   5.42606e-07       4753.63      0.7116      0.7116      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     112       2472.49   3.04346e-09       4782.58      0.3759      0.3759      146   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


...

 log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     871       2538.78   5.66979e-05       17.5846    6.47e-06       0.001     1290  LS failed, Hessian reset 
     899       2538.79   6.34877e-05       7.51086           1           1     1324   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     924       2538.79   6.93367e-06       5.81726   9.506e-07       0.001     1403  LS failed, Hessian reset 
     929       2538.79   1.22664e-06       6.51431      0.2207           1     1409   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -10.8767
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2328.78   0.000311611       50.7148      0.1814      0.1814      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       

100%|██████████| 1/1 [00:02<00:00,  2.35s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 1 forecasts with cutoffs between 2020-07-31 00:00:00 and 2020-07-31 00:00:00


Initial log joint probability = -8.2067
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2535.31     0.0029917       69.5246           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2537.26     0.0130673       86.7031           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2537.88    0.00131186       14.9721           1           1      350   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     386       2538.26   9.35052e-05       20.9288   5.706e-06       0.001      492  LS failed, Hessian reset 
     399       2538.28    0.00164712       10.2525      0.3426           1      508   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     410        2538.3   6.58474e-05     

Initial log joint probability = -10.8767
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2328.86     0.0058581       30.1359           1           1      115   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2331.24   0.000806484       64.6834      0.2925      0.2925      231   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     251       2331.38   4.99932e-05       14.8747   1.788e-06       0.001      332  LS failed, Hessian reset 
     299       2331.51   8.13314e-05        8.3229      0.5611      0.5611      391   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2331.67     0.0121779       6.02354       0.904           1      509   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     401       2331.67   8.69848e-05    

100%|██████████| 1/1 [00:02<00:00,  2.66s/it]


In [105]:
# final PROPHET
fb_model = Prophet(
    changepoint_prior_scale=tuning_results.loc[0, 'changepoint_prior_scale'],
    seasonality_prior_scale=tuning_results.loc[0, 'seasonality_prior_scale'],
)

fb_model.fit(fb_df)
fb_fc = fb_model.predict(fb_test)
mape_PROPHET = round(mean_absolute_percentage_error(fb_test['y'], fb_fc['yhat']), 3)
mape_PROPHET

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -16.0885
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2660.69   8.83443e-07       4586.67      0.5836      0.5836      126   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     136       2660.71   8.10305e-07       4933.35   1.748e-10       0.001      223  LS failed, Hessian reset 
     151       2660.71    8.0074e-09       4188.87      0.1957      0.1957      242   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


0.326

In [106]:
res_fb = pd.DataFrame()
# LOOP all stores
for store in df_store['store_id'].unique():  # print(store)
    df_data = df_store[df_store['store_id'] == store].set_index('date')[
        ['sales']]
    fb_df = pd.concat([df_data, ts_holiday], axis=1)
    fb_df['sales'] = fb_df['sales']/1e6
    fb_df['holiday'] = fb_df['holiday'].fillna(False).astype('bool')
    fb_df = fb_df.dropna().reset_index().rename(
        {'date': 'ds', 'sales': 'y'}, axis=1)
    fb_train = fb_df.iloc[:-steps_ahead]
    fb_test = fb_df.iloc[-steps_ahead:]

    fb_model = Prophet(
        changepoint_prior_scale=tuning_results.loc[0,'changepoint_prior_scale'],
        seasonality_prior_scale=tuning_results.loc[0,'seasonality_prior_scale'],
    )
    fb_model.add_regressor('holiday')
    fb_model.fit(fb_train)
    fb_fc = fb_model.predict(fb_test)

    # metrics
    fb_mae_OOS = round(mean_absolute_error(fb_test['y'], fb_fc['yhat']))
    fb_mape_OOS = round(mean_absolute_percentage_error(
        fb_test['y'], fb_fc['yhat']), 3)

    print(f'Store {store}')
    print(f'MAPE: {fb_mape_OOS}',
          f'MAE: {fb_mae_OOS}\n',
          sep='\n')

    res_fb = res_fb.append({
        'store_id': store,
        'fb_fc': fb_fc['yhat'],
        'fb_mape_OOS': fb_mape_OOS,
        'fb_mae_OOS': fb_mae_OOS,
    }, ignore_index=True)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -7.54053
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2523.38   9.64828e-07        5092.5      0.3498           1      126   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     118       2523.44   4.00178e-06       5059.71   7.933e-10       0.001      189  LS failed, Hessian reset 
     138       2523.46   1.34002e-06       5077.97   2.629e-10       0.001      249  LS failed, Hessian reset 
     158       2523.47   4.98696e-09       4987.86      0.3865      0.3865      270   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 307222
MAPE: 0.454
MAE: 14

Initial log joint probability = -8.42503
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2375.85   1.63755e-05       5089.27      0.8049      0.8049      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     146       2375.93   1.31589e-07       5043.41   2.625e-11       0.001      227  LS failed, Hessian reset 
     152       2375.93   6.56576e-09       4944.56      0.3922      0.3922      237   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


 65%|██████▍   | 33/51 [12:55<07:03, 23.50s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Store 536898
MAPE: 0.53
MAE: 4

Initial log joint probability = -76.5168
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -76.5168             0        705.35       1e-12       0.001       23   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -579.314
Iteration  1. Log joint probability =   -58.6238. Improved by 520.69.
Iteration  2. Log joint probability =    5.27962. Improved by 63.9034.
Iteration  3. Log joint probability =    374.976. Improved by 369.697.
Iteration  4. Log joint probability =    1175.39. Improved by 800.411.
Iteration  5. Log joint probability =    1176.39. Improved by 1.00558.
Iteration  6. Log joint probability =     1176.9. Improved by 0.502934.
Iteration  7. Log joint probability =    1178.36. Improved by 1.46477.
Iteration  8. Log joint probability =    1179.55. Improved by 1.18988.
Iteration  9. Log

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 536902
MAPE: 0.746
MAE: 6

Initial log joint probability = -35.7429
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -35.7429             0       639.747       1e-12       0.001       25   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -486.16
Iteration  1. Log joint probability =    244.572. Improved by 730.732.
Iteration  2. Log joint probability =    873.334. Improved by 628.762.
Iteration  3. Log joint probability =    1106.31. Improved by 232.978.
Iteration  4. Log joint probability =     1147.1. Improved by 40.787.
Iteration  5. Log joint probability =     1151.8. Improved by 4.69564.
Iteration  6. Log joint probability =    1169.06. Improved by 17.263.
Iteration  7. Log joint probability =    1170.22. Improved by 1.16297.
Iteration  8. Log joint probability =    1170.39. Improved by 0.16808.
Iteration  9. Log j

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 566790
MAPE: 0.724
MAE: 25

Initial log joint probability = -12.9974
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -12.9974             0       682.003       1e-12       0.001       28   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -489.145
Iteration  1. Log joint probability =    630.462. Improved by 1119.61.
Iteration  2. Log joint probability =    1054.99. Improved by 424.531.
Iteration  3. Log joint probability =    1139.87. Improved by 84.8755.
Iteration  4. Log joint probability =    1151.57. Improved by 11.6973.
Iteration  5. Log joint probability =    1178.21. Improved by 26.6484.
Iteration  6. Log joint probability =    1179.14. Improved by 0.926685.
Iteration  7. Log joint probability =     1187.2. Improved by 8.05374.
Iteration  8. Log joint probability =    1188.49. Improved by 1.29179.
Iteration  9. 

In [111]:
# Save results
res_fb.to_pickle('results/res_fb_noPromo.pkl')
# res_fb = pd.read_pickle('results/res_fb_noPromo.pkl')

## FB Prophet with Promotion

In [113]:
fb_df = pd.concat([df_company[['sales']], ts_holiday], axis=1)
fb_df['sales'] = fb_df['sales']/1e6
#fb_df['holiday'] = fb_df['holiday'].fillna(False).astype('bool')
fb_df = fb_df.reset_index().rename({'date': 'ds', 'sales': 'y'}, axis=1)
fb_train = fb_df.iloc[:-steps_ahead]
fb_test = fb_df.iloc[-steps_ahead:]

In [118]:
import datetime

holiday = pd.DataFrame(holidays.Vietnam(years=[2018, 2019, 2020, 2021]).items()).rename({0:'date', 1:'holiday_neg'}, axis=1)
# Add Tet promotion
promo = pd.DataFrame(holiday[holiday.holiday_neg == 'Vietnamese New Year'].date - datetime.timedelta(days=1))
promo['promotion'] = "1 days before Tet Holiday"
# Add Black Friday
promo = promo.append(
            pd.DataFrame({'date':[datetime.date(2020,11,27), datetime.date(2019,11,29),datetime.date(2018,11,23)],
                          'promotion':["Black Friday","Black Friday","Black Friday"]})
        )

#holiday['promotion'] = np.NaN
# All together
#holiday = holiday.append(promo).sort_values('date')

# set to 1 if holiday affect sales negatively
holiday_off = holiday.replace({'Vietnamese New Year.*': 1, 
                 '.*day of Tet Holiday': 1,
                 'International Labor Day': 1,
                 '\D': np.NaN}, 
                regex=True).dropna()

promotions = pd.DataFrame({
        'holiday':'big_promotion',
        'ds':promo.date,
        'lower_window': -14, # 2 weeks before
        'upper_window': 0,
})

holiday_negative = pd.DataFrame({
        'holiday':'big_promotion',
        'ds':holiday_off.date,
        'lower_window': 0,
        'upper_window': 0,
})

holidays_fb = pd.concat((promotions, holiday_negative))

In [119]:
# Grid search set up
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

all_params = [dict(zip(param_grid.keys(), v))
              for v in itertools.product(*param_grid.values())]

In [122]:
mape_PROPHET = []
for params in all_params:
    params['weekly_seasonality']=True
    params['holidays'] = holidays_fb
    # set up model
    m = Prophet(**params)
    #m.add_regressor('holiday')
    m.fit(fb_train)
    # set up CV
    df_cv = cross_validation(
        m,
        initial=(str(fb_train.shape[0]-steps_ahead*2)+' days'),
        period=(str(steps_ahead)+' days'),
        horizon=(str(steps_ahead)+' days'),
        )
    # evaluate
    df_p = performance_metrics(df_cv)
    mape_PROPHET.append(df_p['mape'].values[-1])

# Find the best parameters
tuning_results_withPromo = pd.DataFrame(all_params)
tuning_results_withPromo['mape'] = mape_PROPHET
tuning_results_withPromo = tuning_results_withPromo.sort_values('mape').reset_index(drop=True)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 1 forecasts with cutoffs between 2020-07-31 00:00:00 and 2020-07-31 00:00:00


Initial log joint probability = -8.2067
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2493.63   0.000121767       5091.87           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     148       2493.84    4.3081e-09       5014.53      0.5624      0.5624      188   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


  0%|          | 0/1 [00:00<?, ?it/s]

Initial log joint probability = -10.8767
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      93        2254.9   2.54256e-09       4851.29      0.1519      0.1519      116   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


100%|██████████| 1/1 [00:02<00:00,  2.08s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 1 forecasts with cutoffs between 2020-07-31 00:00:00 and 2020-07-31 00:00:00


...

100%|██████████| 1/1 [00:03<00:00,  3.11s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 1 forecasts with cutoffs between 2020-07-31 00:00:00 and 2020-07-31 00:00:00


Initial log joint probability = -8.2067
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2751.48    0.00455634       134.012           1           1      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199          2753    0.00325489       31.2762           1           1      237   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2753.84    0.00109338       14.4701           1           1      347   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2754.21    0.00106112       28.8122           1           1      458   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     461       2754.57    2.6926e-05       14.2475   1.384e-06       0.001      568  LS failed, Hessian reset 
     499       2754.62   0.000122198     

  0%|          | 0/1 [00:00<?, ?it/s]

Initial log joint probability = -10.8767
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2543.54      0.001695       52.5485           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2547.77    0.00698334       27.3505           1           1      227   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2548.38     0.0128261       38.4085           1           1      345   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2549.05    0.00141221       20.5954       0.263           1      461   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2549.38    0.00106988       27.3713           1           1      607   
    Iter      log prob        ||dx||      ||grad||       alpha  

100%|██████████| 1/1 [00:03<00:00,  3.30s/it]


In [123]:
tuning_results_withPromo

,changepoint_prior_scale,seasonality_prior_scale,weekly_seasonality,holidays,mape
0,0.001,1.00,True,holiday ds lower_window up...,0.057572
1,0.001,0.10,True,holiday ds lower_window up...,0.062468
2,0.500,0.10,True,holiday ds lower_window up...,0.064894
3,0.500,10.00,True,holiday ds lower_window up...,0.065866
4,0.500,1.00,True,holiday ds lower_window up...,0.073379
5,0.001,10.00,True,holiday ds lower_window up...,0.080978
6,0.001,0.01,True,holiday ds lower_window up...,0.121806
7,0.010,0.01,True,holiday ds lower_window up...,0.266949
8,0.010,0.10,True,holiday ds lower_window up...,0.276203
9,0.010,10.00,True,holiday ds lower_window up...,0.292131


In [124]:
# %% final PROPHET
params = {
    'changepoint_prior_scale':tuning_results_withPromo.loc[0, 'changepoint_prior_scale'],
    'seasonality_prior_scale':tuning_results_withPromo.loc[0, 'seasonality_prior_scale'],
    'weekly_seasonality':True,
    'holidays':holidays_fb
}

fb_model = Prophet(**params)

fb_model.fit(fb_df)
fb_fc = fb_model.predict(fb_test)
mape_PROPHET = round(mean_absolute_percentage_error(fb_test['y'], fb_fc['yhat']), 3)
mape_PROPHET

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -16.0885
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2755.15   1.35574e-07       5019.93      0.4741      0.4741      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     113       2755.15   4.23695e-09       4720.58      0.4687      0.4687      141   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


0.287

In [125]:
# LOOP all stores
res_fb = pd.DataFrame()
for store in df_store['store_id'].unique():  # print(store)
    df_data = df_store[df_store['store_id'] == store].set_index('date')[
        ['sales']]
    fb_df = pd.concat([df_data], axis=1)
    fb_df['sales'] = fb_df['sales']/1e6
    fb_df = fb_df.dropna().reset_index().rename(
        {'date': 'ds', 'sales': 'y'}, axis=1)
    fb_train = fb_df.iloc[:-steps_ahead]
    fb_test = fb_df.iloc[-steps_ahead:]

    params = {
        'changepoint_prior_scale': tuning_results_withPromo.loc[0, 'changepoint_prior_scale'],
        'seasonality_prior_scale': tuning_results_withPromo.loc[0, 'seasonality_prior_scale'],
        'weekly_seasonality': True,
        'holidays': holidays_fb
    }

    fb_model = Prophet(**params)
    fb_model.fit(fb_train)
    fb_fc = fb_model.predict(fb_test)

    # metrics
    fb_mae_OOS = round(mean_absolute_error(fb_test['y'], fb_fc['yhat']))
    fb_mape_OOS = round(mean_absolute_percentage_error(
        fb_test['y'], fb_fc['yhat']), 3)

    print(f'Store {store}',
          f'MAPE: {fb_mape_OOS}',
          sep='\n')

    res_fb = res_fb.append({
        'store_id': store,
        'fb_fc': fb_fc['yhat'],
        'fb_mape_OOS': fb_mape_OOS,
        'fb_mae_OOS': fb_mae_OOS,
    }, ignore_index=True)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -7.54053
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2552.76   0.000398074       5196.36      0.7969      0.7969      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     155       2555.73   1.71446e-09       4929.85      0.2616      0.2616      190   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 307222
MAPE: 0.407
Initial log joint probability = -8.42503
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2391.63   1.26207e-07       5162.22      0.6963      0.6963      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     113       2391.63   8.63521e-09       5105.33       0.452       0.452      141   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Store 536898
MAPE: 0.495
Initial log joint probability = -76.5168
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -76.5168             0       705.373       1e-12       0.001       23   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -638.231
Iteration  1. Log joint probability =     341.16. Improved by 979.39.
Iteration  2. Log joint probability =    848.188. Improved by 507.028.
Iteration  3. Log joint probability =    1126.42. Improved by 278.231.
Iteration  4. Log joint probability =    1236.27. Improved by 109.85.
Iteration  5. Log joint probability =    1404.75. Improved by 168.484.
Iteration  6. Log joint probability =    1417.94. Improved by 13.1907.
Iteration  7. Log joint probability =     1448.8. Improved by 30.855.
Iteration  8. Log joint probability =     1449.7. Improved by 0.898102.
Iteration  9. Log joint pr

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 536902
MAPE: 0.716
Initial log joint probability = -35.7429
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -35.7429             0       639.753       1e-12       0.001       25   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -545.077
Iteration  1. Log joint probability =    207.429. Improved by 752.506.
Iteration  2. Log joint probability =     798.01. Improved by 590.581.
Iteration  3. Log joint probability =    1118.38. Improved by 320.374.
Iteration  4. Log joint probability =    1210.89. Improved by 92.507.
Iteration  5. Log joint probability =    1222.74. Improved by 11.8472.
Iteration  6. Log joint probability =    1227.32. Improved by 4.5812.
Iteration  7. Log joint probability =    1231.75. Improved by 4.42969.
Iteration  8. Log joint probability =    1234.18. Improved by 2.42664.
Iteration  9. Log joint pr

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 566790
MAPE: 0.651
Initial log joint probability = -12.9974
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
       1      -12.9974             0       682.007       1e-12       0.001       28   
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -548.062
Iteration  1. Log joint probability =    515.484. Improved by 1063.55.
Iteration  2. Log joint probability =    996.821. Improved by 481.336.
Iteration  3. Log joint probability =    1229.23. Improved by 232.414.
Iteration  4. Log joint probability =    1247.31. Improved by 18.0754.
Iteration  5. Log joint probability =    1248.03. Improved by 0.725035.
Iteration  6. Log joint probability =    1249.48. Improved by 1.44168.
Iteration  7. Log joint probability =    1250.84. Improved by 1.36813.
Iteration  8. Log joint probability =    1251.43. Improved by 0.587382.
Iteration  9. Log join

In [127]:
# Save results
res_fb.to_pickle('results/res_fb_withPromo.pkl')


# result

In [206]:
res_ARIMA = pd.read_pickle('results/res_RW_ARIMA_whole_holiday_promo.pkl')
res_fb_noPromo = pd.read_pickle('results/res_fb_noPromo.pkl')
res_fb_withPromo = pd.read_pickle('results/res_fb_withPromo.pkl')

# rename
res_ARIMA = res_ARIMA.rename({
    'fc_RW':'RW_fc',
    'RW_mae_OOS': 'RW_mae',
    'RW_mape_OOS': 'RW_mape',
    'fc_OOS': 'ARIMA_fc',
    'arima_mae_OOS': 'ARIMA_mae',
    'arima_mape_OOS': 'ARIMA_mape',
    }, axis=1).set_index('store_id')

# rearrange
res_ARIMA = res_ARIMA[[
    'RW_fc',
    'RW_mae',
    'RW_mape',
    'ARIMA_fc',
    'ARIMA_mae',
    'ARIMA_mape',
]]

# rename
res_fb_noPromo = res_fb_noPromo.rename({'fb_fc': 'FB_noPromo_fc',
                                        'fb_mae_OOS': 'FB_noPromo_mae',
                                        'fb_mape_OOS': 'FB_noPromo_mape'},axis=1).set_index('store_id')

# rename
res_fb_withPromo = res_fb_withPromo.rename({'fb_fc': 'FB_withPromo_fc',
                                        'fb_mae_OOS': 'FB_withPromo_mae',
                                        'fb_mape_OOS': 'FB_withPromo_mape'},axis=1).set_index('store_id')

# join
res = res_ARIMA.join([
    res_fb_noPromo,
    res_fb_withPromo])
res.index = res.index.astype('int').astype('str')
res.head()

,RW_fc,RW_mae,RW_mape,ARIMA_fc,ARIMA_mae,ARIMA_mape,FB_noPromo_fc,FB_noPromo_mae,FB_noPromo_mape,FB_withPromo_fc,FB_withPromo_mae,FB_withPromo_mape
store_id,,,,,,,,,,,,
307222,date 2020-11-01 21.99 2020-11-02 21.99 2...,13.0,0.452,date 2020-11-01 26.469510 2020-11-02 20....,13.0,0.428,0 29.634086 1 15.412937 2 18.96985...,14.0,0.454,0 27.760678 1 13.699580 2 17.00901...,13.0,0.407
307244,date 2020-11-01 23.4081 2020-11-02 23.40...,10.0,0.454,date 2020-11-01 19.930375 2020-11-02 20....,10.0,0.461,0 19.462731 1 11.312297 2 13.12938...,11.0,0.452,0 20.572647 1 11.722677 2 13.65801...,12.0,0.477
307248,date 2020-11-01 14.2416 2020-11-02 14.24...,9.0,0.451,date 2020-11-01 20.385643 2020-11-02 13....,7.0,0.374,0 20.255784 1 11.290717 2 12.55305...,7.0,0.382,0 19.989263 1 11.132565 2 12.35890...,7.0,0.370
320264,date 2020-11-01 14.91545 2020-11-02 14.9...,7.0,0.465,date 2020-11-01 19.012270 2020-11-02 13....,8.0,0.520,0 15.689275 1 5.480050 2 7.34717...,7.0,0.492,0 15.098892 1 5.790781 2 7.50287...,7.0,0.458
328165,date 2020-11-01 60.16385 2020-11-02 60.1...,41.0,0.594,date 2020-11-01 86.226162 2020-11-02 35....,37.0,0.549,0 68.981854 1 23.303577 2 26.17...,33.0,0.488,0 67.348080 1 19.809141 2 22.44...,31.0,0.448


### compare across stores

In [305]:
res_mape = res[['RW_mape', 'ARIMA_mape', 'FB_noPromo_mape', 'FB_withPromo_mape']]
res_mape.plot()

In [209]:
res_mape.mean()

RW_mape              0.624921
ARIMA_mape           0.546842
FB_noPromo_mape      0.495474
FB_withPromo_mape    0.481579
dtype: float64

### compare across horizons

In [290]:
res_fc = res[['RW_fc','ARIMA_fc','FB_noPromo_fc','FB_withPromo_fc']]
df_test = df_store.iloc[:,:3].set_index('date').pivot(columns='store_id', values='sales')[-steps_ahead:].T/1e6
df_test.index = df_test.index.astype('str')


In [316]:
# plot forecasts & true values
# idx = df_test.index[1]
idx = '307248'
true = df_test.loc[idx, :]
fc = res_fc.loc[idx, :]

fig = go.Figure()
# true data
fig.add_trace(
    go.Scatter(
        x=true.index,
        y=true,
        mode='lines', name='y_true', line=dict(color='black')
    ))
# forecast
for i in range(0, 4):
    fig.add_trace(
        go.Scatter(
            x=true.index,
            y=fc.iloc[i].values,
            mode='lines', name=res_fc.columns[i]),
    )
fig.show()


In [317]:
df_store.loc[df_store.store_id == int(idx), ['date','sales']].set_index('date').plot()